In [8]:
import tkinter as tk
from tkinter import messagebox

# Node and Queue classes
class Node:
    def __init__(self, name, people):
        self.name = name
        self.people = people
        self.next = None

class Queue:
    def __init__(self):
        self.head = None

    def add_to_queue(self, name, people):
        new_node = Node(name, people)
        if self.head is None:
            self.head = new_node
        else:
            node = self.head
            while node.next:
                node = node.next
            node.next = new_node

    def remove_from_queue(self):
        if self.head is None:
            return None
        else:
            node = self.head
            self.head = node.next
            return node  # return the dequeued node

    def is_empty(self):
        return self.head is None

    def print_queue(self):
        if self.head is None:
            return "Waitlist is empty."
        else:
            node = self.head
            waitlist = []
            while node:
                waitlist.append(f"{node.name} (Party of {node.people})")
                node = node.next
            return "\n".join(waitlist)

# Restaurant class with queue and booking logic
class Restaurant:
    def __init__(self, total_tables):
        self.total_tables = total_tables
        self.bookings = {}  # Dictionary to store bookings {table_number: (customer_name, people)}
        self.waitlist = Queue()  # Queue for customers in waitlist

    def reserve_table(self, customer_name, people):
        available_tables = [table for table in range(1, self.total_tables + 1) if table not in self.bookings]
        if available_tables:
            table_number = available_tables[0]
            self.bookings[table_number] = (customer_name, people)
            return f"Table {table_number} reserved for {customer_name} (Party of {people})."
        else:
            self.waitlist.add_to_queue(customer_name, people)
            return f"No available tables. {customer_name} added to the waitlist."

    def cancel_reservation(self, table_number):
        if table_number in self.bookings:
            customer_name, people = self.bookings.pop(table_number)
            if not self.waitlist.is_empty():
                next_customer = self.waitlist.remove_from_queue()
                self.bookings[table_number] = (next_customer.name, next_customer.people)
                return f"Reservation for {customer_name} canceled. Table {table_number} reassigned to {next_customer.name} (Party of {next_customer.people})."
            else:
                return f"Reservation for {customer_name} canceled."
        else:
            return f"Table {table_number} is not currently reserved."

    def display_reservations(self):
        if self.bookings:
            result = []
            for table, (customer, people) in self.bookings.items():
                result.append(f"Table {table}: {customer} (Party of {people})")
            return "\n".join(result)
        else:
            return "No current reservations."

    def display_waitlist(self):
        return self.waitlist.print_queue()

# GUI Implementation
class RestaurantApp:
    def __init__(self, root):
        self.restaurant = Restaurant(total_tables=5)

        root.title("Restaurant Reservation System")
        root.geometry("400x300")

        # Labels and Entry fields for reservation
        self.name_label = tk.Label(root, text="Customer Name:")
        self.name_label.pack()
        self.name_entry = tk.Entry(root)
        self.name_entry.pack()

        self.people_label = tk.Label(root, text="Number of People:")
        self.people_label.pack()
        self.people_entry = tk.Entry(root)
        self.people_entry.pack()

        # Buttons for Booking, Canceling, and Showing reservations
        self.reserve_button = tk.Button(root, text="Reserve Table", command=self.reserve_table)
        self.reserve_button.pack()

        self.cancel_label = tk.Label(root, text="Enter Table Number to Cancel:")
        self.cancel_label.pack()
        self.cancel_entry = tk.Entry(root)
        self.cancel_entry.pack()

        self.cancel_button = tk.Button(root, text="Cancel Reservation", command=self.cancel_reservation)
        self.cancel_button.pack()

        self.show_reservations_button = tk.Button(root, text="Show Current Reservations", command=self.show_reservations)
        self.show_reservations_button.pack()

        self.show_waitlist_button = tk.Button(root, text="Show Waitlist", command=self.show_waitlist)
        self.show_waitlist_button.pack()

        # Text area for displaying results
        self.result_text = tk.Text(root, height=10, width=40)
        self.result_text.pack()

    def reserve_table(self):
        name = self.name_entry.get()
        people = self.people_entry.get()
        if not name or not people:
            messagebox.showerror("Input Error", "Please enter both name and number of people.")
        else:
            try:
                people = int(people)
                result = self.restaurant.reserve_table(name, people)
                self.result_text.delete(1.0, tk.END)
                self.result_text.insert(tk.END, result)
            except ValueError:
                messagebox.showerror("Input Error", "Number of people must be an integer.")

    def cancel_reservation(self):
        table_number = self.cancel_entry.get()
        if not table_number:
            messagebox.showerror("Input Error", "Please enter a table number to cancel.")
        else:
            try:
                table_number = int(table_number)
                result = self.restaurant.cancel_reservation(table_number)
                self.result_text.delete(1.0, tk.END)
                self.result_text.insert(tk.END, result)
            except ValueError:
                messagebox.showerror("Input Error", "Table number must be an integer.")

    def show_reservations(self):
        result = self.restaurant.display_reservations()
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, result)

    def show_waitlist(self):
        result = self.restaurant.display_waitlist()
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, result)

# Main loop to run the GUI
if __name__ == "__main__":
    root = tk.Tk()
    app = RestaurantApp(root)
    root.mainloop()
